# Imports

In [1]:
import os
import numpy as np

from tqdm import tqdm

import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

In [2]:
from Nim.Nim import Nim
from Nim.NimLogic import NimLogic

from Agents.MinimaxAgent import MinimaxAgent

Random seed for reproducibility

In [3]:
np.random.seed(42)

# Agent Testing Function

In [4]:
def test_agent(_misereAgent, _normalAgent, _misere, _initial_piles):


    explored_nodes = np.ndarray(EPISODES)
    moves_count = np.ndarray(EPISODES)
    mean_nodes = np.ndarray(EPISODES)

    for i in tqdm(range(EPISODES)):
        game = Nim(
            initial_piles=_initial_piles[i],
            misere=_misere[i]
        )

        agent = _misereAgent if _misere[i] else _normalAgent

        winner = game.play(
            player1=agent,
            player2=agent,
            verbose=False
        )

        """ AGENT VALIDATION """
        assert winner == NimLogic.is_p_position(_initial_piles[i], _misere[i])

        agent.compute_mean_nodes()

        mean_nodes[i] = agent.mean_nodes
        moves_count[i] = agent.moves_count
        explored_nodes[i] = agent.nodes_explored

    weighted_mean_nodes = explored_nodes.sum() / moves_count.sum()

    avg_explored_nodes = explored_nodes.mean()
    avg_moves_count = moves_count.mean()

    values = [weighted_mean_nodes, avg_explored_nodes, avg_moves_count]

    labels = [
        "average explored nodes per move:",
        "average explored nodes per game:",
        "average moves per game:"
    ]

    label_width = max(len(lbl) for lbl in labels)

    for lbl, val in zip(labels, values):
        print(f"{lbl:<{label_width}} {val:>10.2f}")

    return explored_nodes, moves_count, mean_nodes

# Test running function

In [5]:
def run_tests(misere_agents, normal_agents, max_depth, pile_count, max_pile, episodes, save_path_prefix):
    os.makedirs(save_path_prefix, exist_ok=True)

    print("-" * 60)
    classic_configuration = False

    if pile_count == 0 or max_pile == 0:
        classic_configuration = True
        initial_piles = np.tile([1, 3, 5, 7], (episodes, 1))

        print("Classic configuration used")
    else:
        initial_piles = np.random.randint(1, max_pile, size=(episodes, pile_count))

        print(f"Configuration: pile_count: {pile_count}, max_pile: {max_pile}, max_depth: {max_depth}")

    print("-" * 60)
    misere_modes = np.random.choice([True, False], size=episodes)

    for agent_key in misere_agents.keys():
        if not classic_configuration:
            filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-{pile_count}-{max_pile}.npz"
        else:
            filename = f"{save_path_prefix}/minimax-{agent_key}-{max_depth}-classic.npz"

        if os.path.exists(filename):
            print(f"Skipping {filename}")
            continue

        misere_agent = misere_agents[agent_key]
        normal_agent = normal_agents[agent_key]
        result = test_agent(misere_agent, normal_agent, misere_modes, initial_piles)

        np.savez(filename, result)


# Plotting results

In [6]:
def plot_results(vary_pile_counts, vary_max_piles,
                           max_depth, folder="../savedData/Minimax",
                           save_folder="../savedPlots/Minimax"):
    os.makedirs(save_folder, exist_ok=True)

    models     = ['n',  'c',  'p',  'a',  'cp',  'ca',  'pa',  'cpa']
    model_names = ['Normal',
                   'Canonical',
                   'P-pruning',
                   'Aggressive',
                   'Canon+P-prune',
                   'Canon+Aggressive',
                   'P-prune+Aggressive',
                   'All three']

    cmaps = ['Blues', 'Greens', 'Reds']
    metric_labels = ['Nodes/Move', 'Nodes/Game', 'Moves/Game']

    n_agents = len(models)
    ncols_outer = 4
    nrows_outer = int(np.ceil(n_agents / ncols_outer))

    fig = plt.figure(figsize=(4*ncols_outer, 4*nrows_outer))
    outer_gs = GridSpec(nrows_outer, ncols_outer, wspace=0.3, hspace=0.3)

    for idx, (agent_key, agent_name) in enumerate(zip(models, model_names)):
        row = idx // ncols_outer
        col = idx % ncols_outer
        inner_gs = outer_gs[row, col].subgridspec(len(vary_pile_counts),
                                                  len(vary_max_piles),
                                                  wspace=0.1, hspace=0.1)

        for i, pc in enumerate(vary_pile_counts):
            for j, mp in enumerate(vary_max_piles):
                ax = fig.add_subplot(inner_gs[i, j])

                fname = f"{folder}/minimax-{agent_key}-{max_depth}-{pc}-{mp}.npz"
                if not os.path.exists(fname):
                    ax.text(0.5, 0.5, 'no data', ha='center', va='center')
                    ax.set_xticks([])
                    ax.set_yticks([])
                    continue

                data = np.load(fname)
                exp_nodes, moves_cnt, mean_nodes = data['arr_0']

                m0 = exp_nodes.sum() / moves_cnt.sum()
                m1 = exp_nodes.mean()
                m2 = moves_cnt.mean()
                metrics = [m0, m1, m2]

                bars = ax.bar([0,1,2], metrics)
                for k, bar in enumerate(bars):
                    cmap = plt.get_cmap(cmaps[k])
                    bar.set_color(cmap((k+1)/4.))

                    ax.text(bar.get_x()+bar.get_width()/2,
                            bar.get_height()+0.02*max(metrics),
                            f"{metrics[k]:.1f}",
                            ha='center', va='bottom', fontsize=6)

                ax.set_xticks([])
                ax.set_yticks([])
                if i==0 and j==0:
                    ax.set_title(agent_name, fontsize=8)

    plt.suptitle(f"Nested Metrics for depth={max_depth}", fontsize=12)
    out = f"{save_folder}/nested_depth_{max_depth}.png"
    fig.savefig(out, dpi=200, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved nested plot to {out}")

In [7]:
EPISODES = 1000

PILE_COUNT = 4
MAX_PILE = 127

In [8]:
for max_depth in [1, 2]:
    misere_agents = {
        'n': MinimaxAgent(misere=True, max_depth=max_depth),
        'c': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=True, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=True, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=True, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    normal_agents = {
        'n': MinimaxAgent(misere=False, max_depth=max_depth),
        'c': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True),
        'p': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True),
        'a': MinimaxAgent(misere=False, max_depth=max_depth, aggressive=True),
        'cp': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True),
        'ca': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, aggressive=True),
        'pa': MinimaxAgent(misere=False, max_depth=max_depth, P_pruning=True, aggressive=True),
        'cpa': MinimaxAgent(misere=False, max_depth=max_depth, canonical=True, P_pruning=True, aggressive=True)
    }

    for pile_count in [2, 4, 8]:
        for max_count in [63, 127, 255]:
            run_tests(misere_agents, normal_agents, max_depth, pile_count, max_count, EPISODES, "../savedData/Minimax")

------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 63, max_depth: 1
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-1-2-63.npz
Skipping ../savedData/Minimax/minimax-c-1-2-63.npz
Skipping ../savedData/Minimax/minimax-p-1-2-63.npz
Skipping ../savedData/Minimax/minimax-a-1-2-63.npz
Skipping ../savedData/Minimax/minimax-cp-1-2-63.npz
Skipping ../savedData/Minimax/minimax-ca-1-2-63.npz
Skipping ../savedData/Minimax/minimax-pa-1-2-63.npz
Skipping ../savedData/Minimax/minimax-cpa-1-2-63.npz
------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 127, max_depth: 1
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-1-2-127.npz
Skipping ../savedData/Minimax/minimax-c-1-2-127.npz
Skipping ../savedData/Minimax/minimax-p-1-2-127.npz
Skipping ../savedData/Minimax/minimax-a-1-2-127.npz
Skipping ../savedDat

In [9]:
for max_depth in [1, 2]:
    plot_results(
        vary_pile_counts=[2, 4, 8],
        vary_max_piles=[63, 127, 255],
        max_depth=max_depth,
    )

Saved nested plot to ../savedPlots/Minimax/nested_depth_1.png
Saved nested plot to ../savedPlots/Minimax/nested_depth_2.png


# Max Depth testing on Smaller Configuration

In [10]:
EPISODES = 1000

PILE_COUNT = 3
MAX_PILE = 5

MAX_DEPTH = 9999

In [11]:
misere_agents = {
    'n': MinimaxAgent(misere=True, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=True, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

normal_agents = {
    'n': MinimaxAgent(misere=False, max_depth=MAX_DEPTH),
    'c': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True),
    'p': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True),
    'a': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, aggressive=True),
    'cp': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True),
    'ca': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, aggressive=True),
    'pa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, P_pruning=True, aggressive=True),
    'cpa': MinimaxAgent(misere=False, max_depth=MAX_DEPTH, canonical=True, P_pruning=True, aggressive=True)
}

for pile_count in [2, 3, 4]:
    for max_pile in [3, 4, 5]:
        run_tests(misere_agents, normal_agents, MAX_DEPTH, pile_count, max_pile, EPISODES, "../savedData/Minimax")

------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 3, max_depth: 9999
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-c-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-p-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-a-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-cp-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-ca-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-pa-9999-2-3.npz
Skipping ../savedData/Minimax/minimax-cpa-9999-2-3.npz
------------------------------------------------------------
Configuration: pile_count: 2, max_pile: 4, max_depth: 9999
------------------------------------------------------------


100%|██████████| 1000/1000 [00:00<00:00, 4380.52it/s]


average explored nodes per move:      19.20
average explored nodes per game:      63.84
average moves per game:                3.33


100%|██████████| 1000/1000 [00:00<00:00, 12887.98it/s]


average explored nodes per move:      17.22
average explored nodes per game:      57.25
average moves per game:                3.33


...

average explored nodes per move:     454.06
average explored nodes per game:    3147.08
average moves per game:                6.93


100%|██████████| 1000/1000 [00:00<00:00, 3726.45it/s]


average explored nodes per move:      11.87
average explored nodes per game:      57.08
average moves per game:                4.81


100%|██████████| 1000/1000 [00:00<00:00, 5583.22it/s]

average explored nodes per move:       9.53
average explored nodes per game:      46.34
average moves per game:                4.86
------------------------------------------------------------
Configuration: pile_count: 4, max_pile: 5, max_depth: 9999
------------------------------------------------------------
Skipping ../savedData/Minimax/minimax-n-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-c-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-p-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-a-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-cp-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-ca-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-pa-9999-4-5.npz
Skipping ../savedData/Minimax/minimax-cpa-9999-4-5.npz


In [12]:
plot_results(
    vary_pile_counts=[2, 3, 4],
    vary_max_piles=[3, 4, 5],
    max_depth=MAX_DEPTH
)

Saved nested plot to ../savedPlots/Minimax/nested_depth_9999.png
